## Splunk configuration (Distributed Clustered Deployment)

`cat /opt/splunk/etc/splunk-launch.conf`

> SPLUNK_HOME=/opt/splunk

Перезагрузить: 
`$SPLUNK_HOME/bin/splunk restart`

**1. Search Head 1 (Cluster)**

**2. Search Head 2 (Cluster)**

**3. Cluster Master + Deployer**

* Создать новый индекс: `$SPLUNK_HOME/etc/master-apps/_cluster/local` в `indexes.conf`
```
[wineventlog]
repFactor = auto
homePath = $SPLUNK_DB/wineventlog/db
coldPath = $SPLUNK_DB/wineventlog/colddb
thawedPath = $SPLUNK_DB/wineventlog/thaweddb
maxTotalDataSizeMB = <>
#coldToFrozenDir =
#frozenTimePeriodInSecs = 
```

* Настройки кластеров, например, `replication_factor` и `search_factor`; `indexer_discovery`: `$SPLUNK_HOME/etc/system/local` в `server.conf`


```
[license]
master_uri = https://192.168.26.2:8089

[clustering]
cluster_label = idxcluster
mode = master
replication_factor = 1
pass4SymmKey = <>
rebalance_threshold = 0.9
search_factor = 1

[shclustering]
shcluster_label = shcluster
pass4SymmKey = <>

[indexer_discovery]
pass4SymmKey = <>
indexerWeightByDiskCapacity = true
```

**4. Indexer 1 (Cluster)**

**5. Indexer 2 (Cluster)**

Директории с индексами:
`$SPLUNK_DB=$SPLUNK_HOME/var/lib/splunk`

Например, `$SPLUNK_DB/wineventlog` -> `db (warm/hot)`, `colddb`

**6. Heavy Forwarder**

> Splunk DB Connect

Драйвера для СУБД:

`$SPLUNK_HOME/etc/apps/splunk_app_db_connect/drivers`

Настройки: `/opt/splunk/etc/apps/splunk_app_db_connect/local`

> Data inputs (Например, TCP/UDP -> syslog)

* Подключение источников (без Deployment Server):

`$SPLUNK_HOME/etc/system/local` в `inputs.conf`:
```
[udp://172.30.1.183,172.30.0.10:514]
disabled = 0
sourcetype = VCENTER:syslog
#acceptFrom = 172.30.1.183,172.30.0.10
connection_host = ip
index = vcenter

[udp://<domain_name>:514]
disabled = 0
index = skdpu
sourcetype = skdpu:syslog
connection_host = dns
```

* Подключение инстанса к Deployment Server:

`$SPLUNK_HOME/etc/system/local` в `deploymentclient.conf`:
```
[target-broker:deploymentServer]
targetUri = 192.168.26.8:8089
```

* Приложения:

`$SPLUNK_HOME/etc/apps`

* Подключение источников (с Deployment Server):

`$SPLUNK_HOME/etc/apps/<app_name>/local` в `inputs.conf`:
```
[udp://172.30.0.10:514]
disabled = 0
connection_host = ip
index = vcenter
sourcetype = vcenter:syslog
```

* Извлечение определенных событий: 

`props.conf`:
```
[skdpu:syslog] <- sourcetype
disabled = false
TRANSFORMS-set = setnull, setparsing
```

`transforms.conf`:
```
[setnull]
REGEX= .
DEST_KEY = queue
FORMAT = nullQueue

[setparsing]
REGEX = audit
DEST_KEY = queue
FORMAT = indexQueue
```

**7. Deployment Server**

* To create a new deployment app:

`$SPLUNK_HOME/etc/deployment-apps/` -> `<app_name>` -> `local`

* To push a new version in case of modification a deployment app:

`$SPLUNK_HOME/bin/splunk reload deploy-server`

**8. Universal Forwarders (агенты)**

Security log:

`SplunkUniversalForwarder/etc/apps/<app_name>/local` в `inputs.conf`:

```
[WinEventLog://Security]
disabled = 0
index = wineventlog
```

`SplunkUniversalForwarder/etc/apps/<app_name>/local` в `outputs.conf`:

```
[indexer_discovery:master]
pass4SymmKey = <>
master_uri = https://192.168.26.9:8089

[tcpout:group1]
indexerDiscovery = master
```